In [1]:
import pandas as pd
import numpy as np
import math

In [21]:
gamesdf = pd.read_csv("../resources/preclean/clean_games.csv", parse_dates=["SEASON"])
gamesdf = gamesdf.drop(gamesdf[["Unnamed: 0"]], axis=1)
gamesdf["GAME_DATE_EST"] = pd.to_datetime(gamesdf.GAME_DATE_EST)
gamesdf.dtypes

GAME_DATE_EST      datetime64[ns]
GAME_ID                     int64
HOME_TEAM_ID                int64
VISITOR_TEAM_ID             int64
SEASON             datetime64[ns]
TEAM_ID_home                int64
PTS_home                  float64
FG_PCT_home               float64
FT_PCT_home               float64
FG3_PCT_home              float64
AST_home                  float64
REB_home                  float64
TEAM_ID_away                int64
PTS_away                  float64
FG_PCT_away               float64
FT_PCT_away               float64
FG3_PCT_away              float64
AST_away                  float64
REB_away                  float64
HOME_TEAM_WINS              int64
dtype: object

In [3]:
detailsdf = pd.read_csv("../resources/preclean/clean_games_details.csv")
detailsdf[["MIN", "SECOND"]] = detailsdf["MIN"].str.split(":", expand=True)
detailsdf = detailsdf.drop(detailsdf[["Unnamed: 0", "SECOND"]], axis=1)
detailsdf =  detailsdf.dropna()
detailsdf.isnull().sum()

/Users/mitch/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


GAME_ID        0
TEAM_ID        0
PLAYER_ID      0
PLAYER_NAME    0
MIN            0
FGM            0
FGA            0
FG_PCT         0
FG3M           0
FG3A           0
FG3_PCT        0
FTM            0
FTA            0
FT_PCT         0
REB            0
AST            0
STL            0
BLK            0
TO             0
PTS            0
dtype: int64

In [4]:
detailsdf

,GAME_ID,TEAM_ID,PLAYER_ID,PLAYER_NAME,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,REB,AST,STL,BLK,TO,PTS
0,22200477,1610612759,1629641,Romeo Langford,18,1.0,1.0,1.000,0.0,0.0,0.000,0.0,0.0,0.000,2.0,0.0,1.0,0.0,2.0,2.0
1,22200477,1610612759,1631110,Jeremy Sochan,31,7.0,14.0,0.500,2.0,4.0,0.500,7.0,10.0,0.700,9.0,6.0,1.0,0.0,2.0,23.0
2,22200477,1610612759,1627751,Jakob Poeltl,21,6.0,9.0,0.667,0.0,0.0,0.000,1.0,1.0,1.000,4.0,1.0,1.0,0.0,2.0,13.0
3,22200477,1610612759,1630170,Devin Vassell,30,4.0,13.0,0.308,1.0,6.0,0.167,1.0,1.0,1.000,9.0,5.0,3.0,0.0,2.0,10.0
4,22200477,1610612759,1630200,Tre Jones,27,7.0,12.0,0.583,1.0,3.0,0.333,4.0,4.0,1.000,2.0,3.0,0.0,0.0,2.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557051,11200086,1610612747,2768,Chris Duhon,8,1.0,3.0,0.333,0.0,1.0,0.000,0.0,0.0,0.000,1.0,2.0,1.0,0.0,0.0,2.0
557052,11200086,1610612747,202365,Devin Ebanks,15,2.0,5.0,0.400,0.0,2.0,0.000,0.0,0.0,0.000,1.0,1.0,0.0,0.0,1.0,4.0
557053,11200086,1610612747,2730,Dwight Howard,33,8.0,12.0,0.667,0.0,0.0,0.000,3.0,8.0,0.375,12.0,2.0,0.0,4.0,5.0,19.0
557054,11200086,1610612747,201975,Jodie Meeks,15,2.0,4.0,0.500,2.0,2.0,1.000,0.0,0.0,0.000,1.0,1.0,0.0,0.0,0.0,6.0


In [5]:
rankingdf = pd.read_csv("../resources/preclean/clean_ranking.csv", parse_dates=["DATE"])
rankingdf = rankingdf.drop(rankingdf[["Unnamed: 0"]], axis=1)
rankingdf.head()

,TEAM_ID,CONFERENCE,TEAM,G,W,L,W_PCT,DATE
0,1610612743,West,Denver,30,19,11,0.633,2022-12-22
1,1610612763,West,Memphis,30,19,11,0.633,2022-12-22
2,1610612740,West,New Orleans,31,19,12,0.613,2022-12-22
3,1610612756,West,Phoenix,32,19,13,0.594,2022-12-22
4,1610612746,West,LA Clippers,33,19,14,0.576,2022-12-22


In [6]:
rankingdf.dtypes

TEAM_ID                int64
CONFERENCE            object
TEAM                  object
G                      int64
W                      int64
L                      int64
W_PCT                float64
DATE          datetime64[ns]
dtype: object

In [7]:
playersdf = pd.read_csv("../resources/preclean/players.csv", parse_dates=["SEASON"])
playersdf.dtypes

PLAYER_NAME            object
TEAM_ID                 int64
PLAYER_ID               int64
SEASON         datetime64[ns]
dtype: object

In [8]:
playersdf["SEASON"] = playersdf["SEASON"].dt.to_period("Y")
playersdf

,PLAYER_NAME,TEAM_ID,PLAYER_ID,SEASON
0,Royce O'Neale,1610612762,1626220,2019
1,Bojan Bogdanovic,1610612762,202711,2019
2,Rudy Gobert,1610612762,203497,2019
3,Donovan Mitchell,1610612762,1628378,2019
4,Mike Conley,1610612762,201144,2019
...,...,...,...,...
7223,Lanny Smith,1610612758,201831,2009
7224,Warren Carter,1610612752,201999,2009
7225,Bennet Davis,1610612751,201834,2009
7226,Brian Hamilton,1610612751,201646,2009


In [9]:
playersdf.dtypes

PLAYER_NAME           object
TEAM_ID                int64
PLAYER_ID              int64
SEASON         period[A-DEC]
dtype: object

In [10]:
gamesdf.to_csv("../resources/games.csv", index=False)
detailsdf.to_csv("../resources/game_details.csv", index=False)
rankingdf.to_csv("../resources/ranking.csv", index=False)
playersdf.to_csv("../resources/players.csv", index=False)